## Inlezen Objecttypes

Hieronder wordt het model (vanaf 'root_guid' en alles daaronder) ingelezen in het dataframe df voor verdere verwerking. Achtereenvolgens worden de volgende stappen genomen:

1. alle packages (in een tree)
2. alle objecten (classes, enumeraties en waardelijsten) met alle waarden.
3. filteren van objecten op de te exporteren waardes

In [20]:
import datetime

### Configuratie
db_uri = "sqlite://///home/jovyan/root/gemeentelijk gegevensmodel EA16 v2.1.qea"

now = datetime.datetime.now()
dt_string = now.strftime("%d%m%Y-%H:%M:%S")
import_objects = f"./input/ggm_objecttypen_gewijzigd_v3.xlsx"
import_attributes = f"./input/ggm_attributen_gewijzigd_v3.xlsx"
import_definitiefixes = f"./input/Definitiefixes.xlsx"


root_guid = '{604AC59D-D038-4e78-A5FA-80E4748DED5B}' #Om te experimenteren: Onderwijs in package ontwikkeling
root_guid = '{45B88627-6F44-4b6d-BE77-3EC51BBE679E}' #Om te experimenteren: Monumenten
root_guid = '{58A5214F-E56C-4707-BE2D-AB36DD6976A3}' #Om te experimenteren: RSGB
root_guid = '{D7FD597E-1F40-48df-AFFC-EA3B5B5D3FBF}' # Root GGM

## Configuratie

Met de waarden hieronder configureer je de export. Met root_guid geef je de root in de repository weer waar naartoe (en alles eronder) geïmporteerd wordt. Op deze manier kun je ook naar deelmodellen importeren, en alles negeren dat niet bij dat deelmodel hoort. Als voorbeeld zijn naast de root van het hele GGM ook Onderwijs en monumenten getoond.  

In [18]:
### Importeer bibliotheken en utils

import os
import pandas as pd
import numpy as np
import json
from IPython.display import JSON as JSONDisplay
import requests
import database
import util
from re import sub
import ast
import re
from treelib import Node, Tree
from datetime import datetime
import math 

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.sql.expression import collate

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Sequence
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import func
import uuid

def generateEAUUID():
    return '{' +  str(uuid.uuid4()).upper() + '}'

def isEmpty(val):
    if not val:
        return True
    elif pd.isna(val):
        return True
    else:
        return False


## Inlezen CSV met import-informatie over objecten

Eerse wordt CVS ingelezen, en daarna wordt de bestaande data merged met de CSV en worden de waarden uit de kolommen samengevoegd 

In [19]:
#Allereerst alle objectgegevens verzamelen en combineren

# alle objecten (packages, classes, enumeraties en waardelijsten)
df_obj = database.get_df_objectsHierar(db_uri, root_guid=root_guid)

#Packages hebben we niet bodig voor export
df_obj = df_obj[df_obj.object_type != 'Package']

df_obj.head(5)

,object_id,object_type,stereotype,name,alias,author,version,objectnote,ea_guid,modifieddate,note,package_id,tree
0,12,Class,proxy,AanwezigeDeelnemer,None,Arjen Brienen,1.0,iemand die meedoet aan eencollege- of raadsver...,{F1E55DC7-0F33-40ea-8713-2E1AC3D7EE8D},2024-04-17 13:34:23,iemand die meedoet aan eencollege- of raadsver...,11,11-9-5-4-2
1,13,Class,None,Agendapunt,None,Arjen Brienen,1.0,Een onderwerp dat in de vergadering wordt beha...,{73FB5212-40ED-40dc-B837-36588840445A},2024-04-17 13:34:24,Een onderwerp dat in de vergadering wordt beha...,11,11-9-5-4-2
2,14,Class,None,Categorie,None,Arjen Brienen,1.0,Categorie waarop leveranciers zich voor de lev...,{B9B82B25-5D7F-4d2b-84F9-4DF74525EEAD},2024-04-17 13:34:24,Categorie waarop leveranciers zich voor de lev...,11,11-9-5-4-2
3,15,Class,None,Collegelid,None,Arjen Brienen,1.0,Iemand die behoort het college van burgemeeste...,{7B9EDDFD-57F7-4ff2-938F-FDFA3B503DA8},2024-04-17 13:34:24,Iemand die behoort het college van burgemeeste...,11,11-9-5-4-2
4,16,Class,None,Raadscommissie,None,Arjen Brienen,1.0,"<font color=""#191919"">Een raadscommissie binne...",{CB27D699-F82B-45ad-823A-B2B51BCAECBA},2024-04-17 13:43:38,"<font color=""#191919"">Een raadscommissie binne...",11,11-9-5-4-2


In [6]:
df_input = pd.read_excel(import_objects)  
df_input.head()
df_input.head(5)

/opt/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,id,alias,archimate_type,author,bron,created,datum_tijd_export,descr,domein_dcat,domein_gemma,...,name_oud,package_id,phase,status,stereotype,synoniemen,toelichting,uri,version,visibility
0,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,Data object,abrienen,NaN,2019-12-18 13:52:42,28062023-11:06:06,Persoon waarmee communicatie plaatsvindt,NaN,NaN,...,Contact,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
1,EAID_5805FA72_C0CE_43c1_A53F_B81166AEFDD4,NaN,Business object,abrienen,NaN,2019-12-18 14:20:03,28062023-11:06:06,Gebouw waar je tegen betaling kunt logeren.,NaN,NaN,...,Hotel,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
2,EAID_669000E9_25D6_4346_B718_516CDB8B88B7,NaN,Business object,abrienen,NaN,2019-12-18 14:09:16,28062023-11:06:06,Verblijf in een hotel,NaN,NaN,...,Hotelbezoek,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
3,EAID_8F05E7BA_E7ED_45d1_9626_C8D7FBDB2F1B,NaN,Business object,abrienen,NaN,2019-12-18 14:19:47,28062023-11:06:06,Locatie waar iets wordt verkocht,NaN,NaN,...,Verkooppunt,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
4,EAID_EB35F5B8_9289_49cc_8DF4_8BD20EC662A9,NaN,Business object,abrienen,https://www.cbs.nl/nl-nl/onze-diensten/methode...,2019-12-18 14:08:27,28062023-11:06:06,"De vraag naar arbeid, te berekenen door de tot...",NaN,NaN,...,Werkgelegenheid,EAPK_DB8BA49F_252E_4044_A93B_8215B6903827,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN


In [11]:
from sqlalchemy import create_engine, select, MetaData, Table, and_, update, func


# Generic function to set object_types
def saveObjectToDatabase(session, ea_guid, row):
    #print(f'row: {row}')
    t_obj = session.query(Object).filter_by(ea_guid=ea_guid).first()
    if t_obj:
        if row['name'] != row['name_oud']:
            setattr(t_obj, 'Name', row['name'])
            print(f"Changed name from {row['name_oud']} to {row['name']}") 
    else:
        print(f'Could not find object with guid {ea_guid}') 
    

            
# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)
metadata = MetaData(bind=None)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True) #Base.prepare(engine, reflect=True)
Object = Base.classes.t_object

# Do transformation
session = Session(engine)
try:
    for index, row in df_input.iterrows():
        ea_guid = '{'+ str(row['id']).replace("EAID_", "").replace("_", "-") +'}'
        #print(f"processing {ea_guid} with name {row['name_oud']}")
        saveObjectToDatabase(session, ea_guid, row)    
except:
    print(f'Foutmelding {row}')
    raise
        
finally:
    session.commit()
    engine.dispose()
    


Changed name from Aanwezige Deelnemer to AanwezigeDeelnemer
Changed name from Aanvraag Leerlingenvervoer to AanvraagLeerlingenvervoer
Changed name from Aanvraagvrijstelling to AanvraagVrijstelling
Changed name from Beschikking Leerlingenvervoer to BeschikkingLeerlingenvervoer
Changed name from Doorgeleiding OM to DoorgeleidingOM
Changed name from HALT-verwijzing to HALTVerwijzing
Changed name from Klacht Leerlingenvervoer to KlachtLeerlingenvervoer
Changed name from Procesverbaal Onderwijs to ProcesverbaalOnderwijs
Changed name from Ziekmelding Leerlingenvervoer to ZiekmeldingLeerlingenvervoer
Could not find object with guid {1E5BFA0A-588A-4610-AE53-6893D61E707A}
Could not find object with guid {3A686277-3FF1-46ee-99E1-AADA02C6BF46}
Changed name from Ouder Of Verzorger to OuderOfVerzorger
Changed name from CROW-Melding to CROWMelding
Changed name from Geo-Object to GEOObject
Changed name from KadastraleMutatie to Kadastralemutatie
Changed name from Kwaliteitscatalogus Openbare Ruimte t

In [10]:
#Allereerst alle objectgegevens verzamelen en combineren

# alle objecten (packages, classes, enumeraties en waardelijsten)
df_obj = database.get_df_objectsHierar(db_uri, root_guid=root_guid)

#Packages hebben we niet bodig voor export
df_obj = df_obj[df_obj.object_type != 'Package']

df_obj[df_obj.name == 'VLogInfo'].head(5)

,object_id,object_type,stereotype,name,alias,author,version,objectnote,ea_guid,modifieddate,note,package_id,tree
68,95,Class,None,VLogInfo,None,Arjen Brienen,1.0,V-log is een open standaard voor datalogging v...,{E4EEBD4D-A41E-4c9d-8099-CDA45BFF0056},2024-04-17 13:34:25,V-log is een open standaard voor datalogging v...,22,22-20-16-4-2


## Inlezen CSV met import-informatie over objecten

Eerse wordt CSV ingelezen, en daarna wordt de bestaande data merged met de CSV en worden de waarden uit de kolommen samengevoegd 

In [12]:
df_attr = pd.read_excel(import_attributes)  
df_attr.head(5)

/opt/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,id,clazz_id,descr,enumeration_id,name,name_oud,primitive,type_class_id
0,EAID_B4F60A84_AFBB_4554_9887_DF1020E99DB8,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,contactsoort,contactsoort,NaN,NaN
1,EAID_DC8B5E9A_144A_4c78_ACAB_908E8DC2C757,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,datum,datum,NaN,NaN
2,EAID_B8A47DC2_CE92_4e1b_949B_BC2212AC341C,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,tekst,tekst,NaN,NaN
3,EAID_dst36212C_BD57_42b6_B51D_E49EA2853F66,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,NaN,NaN,NaN,NaN
4,EAID_dst4AC804_79E6_40a6_992F_C50B4F045C90,EAID_3BF6985C_06AB_4cfc_9A0E_5CC37C224245,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
from sqlalchemy import create_engine, select, MetaData, Table, and_, update, func


# Generic function to set object_types
def saveAttrToDatabase(session, ea_guid, row):
    #print(f'row: {row}')
    t_attr = session.query(Attribute).filter_by(ea_guid=ea_guid).first()
    if t_attr:
        if row['name'] != row['name_oud']:
            setattr(t_attr, 'Name', row['name'])
            print(f"Changed name from {row['name_oud']} to {row['name']}") 
    else:
        print(f"Could not find object with guid {ea_guid} from {row['name_oud']} to {row['name']}") 
    

            
# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)
metadata = MetaData(bind=None)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True)
Attribute = Base.classes.t_attribute

# Do transformation
session = Session(engine)
try:
    for index, row in df_attr.iterrows():
        ea_guid = '{'+ str(row['id']).replace("EAID_", "").replace("_", "-") +'}'
        #print(f"processing {ea_guid} with name {row['name_oud']}")
        saveAttrToDatabase(session, ea_guid, row)    
except:
    print(f'Foutmelding {row}')
    raise
        
finally:
    session.commit()
    engine.dispose()

Could not find object with guid {dst36212C-BD57-42b6-B51D-E49EA2853F66} from nan to nan
Could not find object with guid {dst4AC804-79E6-40a6-992F-C50B4F045C90} from nan to nan
Changed name from aantal kamers to aantalKamers
Could not find object with guid {dst70B853-AEE4-4a58-A791-9FE8BE675DD6} from nan to nan
Changed name from einddatum to datumEinde
Changed name from startdatum to datumStart
Changed name from aantalFulletimeVrouwen to aantalFulltimeVrouwen
Changed name from aantalkassa to aantalKassa
Changed name from BronWVO to bronWVO
Changed name from winkelvloeroppervlak to winkelvloeroppervlakte
Changed name from Wvoklasse to WVOKlasse
Changed name from eindeaanwezigheid to eindeAanwezigheid
Could not find object with guid {dst36E0F8-0D39-41ce-BE5D-9723DCE88E81} from nan to nan
Could not find object with guid {src64B46B-96A6-4f1c-B92A-44B925160E61} from nan to nan
Changed name from aanstellingsdatum to datumAanstelling
Changed name from uittredingsdatum to datumUittreding
Could 

# Volgende stap

In [21]:
df_def = pd.read_excel(import_definitiefixes)  
df_def.head(5)

,id,alias,archimate_type,authentiek,author,bron,created,datum_tijd_export,definitie,name,nullable,package_id,phase,status,stereotype,synoniemen,toelichting,uri,version,visibility
0,EAID_F1E55DC7_0F33_40ea_8713_2E1AC3D7EE8D,NaN,Business object,NaN,Arjen Brienen,NaN,2019-05-16 15:22:56,28062023-11:06:06,iemand die meedoet aan eencollege- of raadsver...,Aanwezige Deelnemer,NaN,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
1,EAID_73FB5212_40ED_40dc_B837_36588840445A,NaN,Data object,NaN,Arjen Brienen,https://www.notuleercentrum.nl/service-voor-no...,2019-05-16 11:55:17,28062023-11:06:06,Een onderwerp dat in de vergadering wordt beha...,Agendapunt,NaN,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
2,EAID_B9B82B25_5D7F_4d2b_84F9_4DF74525EEAD,NaN,Data object,NaN,Arjen Brienen,NaN,2019-05-16 11:44:01,28062023-11:06:06,Categorie waarop leveranciers zich voor de lev...,Categorie,NaN,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
3,EAID_7B9EDDFD_57F7_4ff2_938F_FDFA3B503DA8,NaN,Business object,NaN,Arjen Brienen,NaN,2019-05-16 11:26:04,28062023-11:06:06,Iemand die behoort het college van burgemeeste...,Collegelid,NaN,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN
4,EAID_475E24C9_CC9A_49cc_AB80_8CA0221441E5,NaN,Business object,NaN,Arjen Brienen,https://www.begrippenxl.nl/dso/nl/page/Dossier,2019-05-16 11:44:26,28062023-11:06:06,Samenhangende set gegevens en informatie voor ...,Dossier,NaN,EAPK_9EBFD9E7_BB04_45c9_97BB_426207B40292,1.0,Proposed,NaN,NaN,NaN,NaN,1.0,NaN


In [23]:
from sqlalchemy import create_engine, select, MetaData, Table, and_, update, func


# Generic function to set object_types
def saveObjectToDatabase(session, ea_guid, row):
    #print(f'row: {row}')
    t_obj = session.query(Object).filter_by(ea_guid=ea_guid).first()
    if t_obj:
        if t_obj.Note != row['definitie']:
            setattr(t_obj, 'Note', row['definitie'])
            print(f"Changed definitie for {t_obj.Name} to {row['definitie']}") 
    else:
        print(f'Could not find object with guid {ea_guid}') 
    

            
# engine, suppose it has two tables 'user' and 'address' set up
engine = create_engine(db_uri)
metadata = MetaData(bind=None)

# mapped classes are now created with names by default
# matching that of the table name.
# reflect the tables
Base = automap_base()
Base.prepare(autoload_with=engine, reflect=True) #Base.prepare(engine, reflect=True)
Object = Base.classes.t_object

# Do transformation
session = Session(engine)
try:
    for index, row in df_def.iterrows():
        ea_guid = '{'+ str(row['id']).replace("EAID_", "").replace("_", "-") +'}'
        #print(f"processing {ea_guid} with name {row['name_oud']}")
        saveObjectToDatabase(session, ea_guid, row)    
except:
    print(f'Foutmelding {row}')
    raise
        
finally:
    session.commit()
    engine.dispose()
    


Changed definitie for Raadscommissie to Een raadscommissie binnen de Nederlandse gemeenteraad is een groep raadsleden die zich buigt over specifieke thema's of beleidsonderwerpen om de besluitvorming in de volledige raad voor te bereiden en te ondersteunen.
Changed definitie for Parkeervergunning to Officiele toestemming dat je op een bepaalde plek mag parkeren
Changed definitie for Parkeervlak to Parkeergelegenheid bestemd voor het parkeren van een of meerdere voertuigen direct langs de doorgaande weg gelegen.
Changed definitie for Vlak to Plat, oneindig oppervlak of variëteit zonder enige kromming. 
Changed definitie for Archiefstuk to Bijeengebrachte informatie, ongeacht het medium, die wordt gecreëerd, ontvangen en gearchiveerd door een bureau, een instelling, een organisatie of een individu met het oog op het nakomen van wettelijke verplichtingen of het uitvoeren van zakelijke transacties.(AAT)
Changed definitie for Indeling to Onderwerpen groeperen in samenhangende categorieën.
C